## Cryptography 1 - Week 3 Programming Assignment 
###Brandon Kinman, 5/24/2015###

The problem description for this assignment is provided in the same directory as this .ipynb file.

Let us first start by testing out PyCrypto's SHA256 function.

In [7]:
from Crypto.Hash import SHA256

hash = SHA256.new()
hash.update('secret')
print(hash.digest().encode('hex'))

2bb80d537b1da3e38bd30361aa855686bde0eacd7162fef6a25fe97bf527a25b


As you can see, this seems to output the correct results. Let us now open the movie file to which we would like to add authenticity.

In [15]:
fname = '2.mp4'
movieFile = open(fname,'r')

Let us also open a new file to write to, which will contain the original file, but with modification.

In [12]:
hashedOutputFile = open('hashed_output.bin','w')

Now we will create a Generator, which will allow us to loop through the file, one block at a time.

In [21]:
def getBlocksFromFile(file_object,chunk_size=1024):
    while True:
        data = file_object.read(chunk_size)
        if not data:
            break
        yield data

Oh, but wait.... We need to loop through the file from the last block to the first... We will need a different and more complicated generator for this.

In [75]:
import os

def getBlocksFromEndOfFile(file_object,chunk_size=1024):
    num_complete_chunks = os.fstat(file_object.fileno()).st_size/chunk_size
    incomplete_chunk_bytes = os.fstat(file_object.fileno()).st_size - num_chunks*chunk_size
    
    while num_complete_chunks or incomplete_chunk_bytes:
        chunk = ''
        if incomplete_chunk_bytes:
            file_object.seek(chunk_size*num_complete_chunks)
            chunk = file_object.read(incomplete_chunk_bytes)
            incomplete_chunk_bytes = 0
        else:
            file_object.seek(chunk_size*(num_complete_chunks-1))
            chunk = file_object.read(chunk_size)
            num_complete_chunks -= 1
        
        yield chunk

Now we will try to compute the final output:

In [108]:
prevHash = ''

for block in getBlocksFromEndOfFile(movieFile):
    hash = SHA256.new()
    hash.update(block+prevHash)
    prevHash = hash.digest()
    
print(prevHash.encode('hex'))

03c08f4ee0b576fe319338139c045c89c3e8e9409633bea29442e21425006ea8


We can indeed see that the above yeilds the correct result, as given in the problem description. Let us now compute the final result for the other movie file.

In [109]:
movieFile = open('1.mp4','r')

prevHash = ''

for block in getBlocksFromEndOfFile(movieFile):
    hash = SHA256.new()
    hash.update(block+prevHash)
    prevHash = hash.digest()
    
print(prevHash.encode('hex'))

5b96aece304a1422224f9a41b228416028f9ba26b0d1058f400200f06a589949


After submitting the assignment, we can see that this was indeed the correct result.